# Test experimental model

This notebook tests the model used for experimental cells runs correctly on a simple step stimulus.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import time

import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import numpy as np

import shutil
from pprint import pprint

import multimodalfitting as mf

import neuroplotlib as nplt

%matplotlib notebook

In [ ]:
model_name = "experimental" # "hay", "hay_ais", "hay_ais_hillock"
probe_type = "planar"
base_folder = Path("../..")
cell_models_folder = base_folder / "cell_models"

model_folder = cell_models_folder / f"{model_name}_model"
data_folder = base_folder / "data_210916" / "experimental" / "cell1_210301"

In [ ]:
morphology = data_folder / "morphology" / "morphology_corrected.swc"
probe_file = data_folder / "extracellular" / "probe.json"
params_release = "experimental_model/parameters_abd_release.json"

In [ ]:
cell = mf.create_experimental_model(morphology_file=morphology,
                                    cell_model_folder=model_folder,
                                    release=True,
                                    v_init=-70, 
                                    abd=False)
cell_abd = mf.create_experimental_model(morphology_file=morphology,
                                    cell_model_folder=model_folder,
                                    release=True,
                                    v_init=-70, 
                                    abd=True)

In [ ]:
# define a probe (this can also be defined externally)
probe = mf.define_electrode(probe_file=probe_file)

# define a test protocol
protocols = mf.define_test_step_protocol(step_amplitude=0.5, tot_duration=500, step_duration=400, probe=probe)

# instantiate simulator
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)
sim_abd = ephys.simulators.LFPySimulator(cell_abd, cvode_active=True, electrode=probe, mechs_folders=model_folder)

## Check ABD section

In [ ]:
from multimodalfitting.imaging_tools.correct_swc import swc_dtype

In [ ]:
swc_data = np.loadtxt(morphology, dtype=swc_dtype)

In [ ]:
x_c = swc_data["x"] - swc_data["x"][0]
y_c = swc_data["y"] - swc_data["y"][0]

In [ ]:
print("No abd")
cell.freeze({})
cell.instantiate(sim)

In [ ]:
sim.neuron.h.topology()

In [ ]:
ax = nplt.plot_neuron(cell.LFPyCell, plane="xy", color_ais="g", color_axon="r",
                      color_abd="b", exclude_sections=["soma"])

In [ ]:
ax.plot(x_c, y_c, ls="", marker=".", color='r')

In [ ]:
sec_to_show = ["[0].dend[29]", "[0].dend[2]", "[0].dend[0]", "[0].dend[1]",
               "[0].abd"]

for sec in cell.LFPyCell.allseclist:
    name = sec.name()
    if any([sec_ in name for sec_ in sec_to_show]):
        n3d = sec.n3d()
        print(f"{name} - pos0: {[sec.x3d(0), sec.y3d(0)]}, pos0: {[sec.x3d(n3d - 1), sec.y3d(n3d - 1)]}")
        print(sim.neuron.h.parent_connection(sec), sim.neuron.h.parent_section(sec))

In [ ]:
cell.unfreeze({})
cell.destroy(sim)

In [ ]:
print("abd")
cell_abd.freeze({})
cell_abd.instantiate(sim)

In [ ]:
ax = nplt.plot_neuron(cell_abd.LFPyCell, plane="xy", color_ais="g", color_abd="b", exclude_sections=["soma"],
                      detailed=True)

In [ ]:
ax.plot(x_c, y_c, ls="", marker=".", color='r')

In [ ]:
sec_to_show = ["[0].dend[29]", "[0].dend[2]", "[0].dend[0]", "[0].dend[1]",
               "[0].abd", ]

for sec in cell_abd.LFPyCell.allseclist:
    name = sec.name()
    if any([sec_ in name for sec_ in sec_to_show]):
        n3d = sec.n3d()
        print(f"{name} - pos0: {[sec.x3d(0), sec.y3d(0)]}, pos0: {[sec.x3d(n3d - 1), sec.y3d(n3d - 1)]}")
        

In [ ]:
cell_abd.unfreeze({})
cell_abd.destroy(sim)